In [ ]:
!pip install imbalanced-learn

In [ ]:
!pip install -U keras-tuner

In [ ]:
!pip install keras

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import catboost as cb
import keras
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, PReLU, ELU, Dropout, Activation
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner import RandomSearch
import keras_tuner
from keras.wrappers.scikit_learn import KerasClassifier

# Reading input data into dataframe

In [ ]:
df= pd.read_csv('../input/smoking-related-lung-cancers/lung_cancer.csv')

In [ ]:
df

# Understanding the data

# 1- we see that race feature has some missing values
# 2- we see that features i.e. days_to_cancer and stage_of_cancer are imbalanced

In [ ]:
df.info()

# Creating a custom dataframe to see the relationship of target features with each other

In [ ]:
df_sub_no_na = df[(df['days_to_cancer'].isna() == False) & (df['stage_of_cancer'].isna() == False)]

# We can see that both the target features contain some non null values in corresponsing rows

In [ ]:
df_sub_no_na

# Preprocessing Categorical features

In [ ]:
df['gender'].unique()

In [ ]:
df['race'].unique()

In [ ]:
df['race'].value_counts()

In [ ]:
df['smoker'].unique()

# As all the categorical featuers contain nominal data, therefore One Hot Encoding has been applied

In [ ]:
gender = pd.get_dummies(df['gender'], drop_first=True)
race = pd.get_dummies(df['race'], drop_first=True)
smoker = pd.get_dummies(df['smoker'], drop_first=True)

In [ ]:
df = pd.concat([df, gender, race, smoker], axis=1)

In [ ]:
df.drop(['gender', 'race', 'smoker'], axis=1, inplace=True)

In [ ]:
df

# Reseting the index of the resultant dataframe after applying One Hot Encoding

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df

# Dropping the pid feature as it is not needed

In [ ]:
df.drop('pid', inplace=True, axis=1)

In [ ]:
df

# Combining the two features days_to_cancer and stage_of_cancer to make a new target feature 'cancer' containing only binary values. 1 represents a person who has cancer and 0 who does not

In [ ]:
cancer_list = list()
days_cancer = list()

In [ ]:
for index in df.index:
    
    if pd.isna(df['days_to_cancer'][index]) == True:
        days_cancer.append(0)
    if pd.isna(df['stage_of_cancer'][index]) == True:
        cancer_list.append(0)
    if pd.isna(df['stage_of_cancer'][index]) == False:
        cancer_list.append(1)
    if pd.isna(df['days_to_cancer'][index]) == False:
        days_cancer.append(df['days_to_cancer'][index])

In [ ]:
df['cancer'] = cancer_list
df['days_cancer'] = days_cancer

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df.describe(include='all')

# Dropping the days_to_cancer and stage_of_cancer feature as they are no longer needed

In [ ]:
df.drop(['days_to_cancer', 'stage_of_cancer'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
df.info()

# Preprocessing Numerical Features i.e. age

# Plotting a histogram to check the distribution of data

In [ ]:
df.hist(['age', 'days_cancer'])

# We can conclude that age and days_cancer feature does not follow normal distribution

# We can now normalize the column age and days_cancer

# Feture Scaling

# Applying Normalization to age and days_cancer

In [ ]:
min_max_scaler = MinMaxScaler()

min_max_scaled_data = min_max_scaler.fit_transform(df[['age', 'days_cancer']])

# Replacing the old values in the dataframe

In [ ]:
df_normalized = pd.DataFrame(min_max_scaled_data, columns=[['age', 'days_cancer']])

In [ ]:
df[['age', 'days_cancer']] = df_normalized[['age', 'days_cancer']]

In [ ]:
df

In [ ]:
df['days_cancer'].unique()

In [ ]:
df.isnull().sum()

# Splitting the data into training and testing data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cancer',axis=1), df['cancer'], test_size=0.3)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train.shape

# Under Sampling

In [ ]:
from collections import Counter
from imblearn.under_sampling import NearMiss

near_miss = NearMiss(0.8)

X_train_nm, y_train_nm = near_miss.fit_resample(X_train, y_train)

print('Before Sampling {}'.format(Counter(y_train)))
print('After Sampling {}'.format(Counter(y_train_nm)))

In [ ]:
rand_for_classifier = RandomForestClassifier()
rand_for_classifier.fit(X_train_nm, y_train_nm)

y_pred_rf_clf = rand_for_classifier.predict(X_test)

print(confusion_matrix(y_pred_rf_clf, y_test))
print(classification_report(y_pred_rf_clf, y_test))

# We can see that the data is unbalanced for the target feature

In [ ]:
df['cancer'].value_counts()

In [ ]:
count_class_0, count_class_1 = df['cancer'].value_counts()
count_class_0, count_class_1

# Separating the data for 0 and 1 classes

In [ ]:
df_class_0 = df[df['cancer'] == 0]
df_class_1 = df[df['cancer'] == 1]

In [ ]:
df_class_0.shape, df_class_1.shape

# Creating a dataframe with equal number of target column values

In [ ]:
df_class_0_under = df_class_0.sample(count_class_1)

df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

df_test_under.shape

In [ ]:
df_test_under['cancer'].value_counts()

In [ ]:
X = df_test_under.drop('cancer', axis=1)
y = df_test_under['cancer']

# Separating the train and test features

In [ ]:
# stratify = y will ensure that the training and testing samples are balanced for both classes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [ ]:
# both the classes are balanced because we set stratify = y
y_train.value_counts()

# Model Training

# Random Forest Classifier

In [ ]:
rand_forest = RandomForestClassifier()

model_randforest = rand_forest.fit(X_train, y_train)

y_pred = model_randforest.predict(X_test)

# F1 Score on Random Forest Classifier

In [ ]:
f1_score(y_test, y_pred)

# Logistic Regression

In [ ]:
logisticRegr = LogisticRegression(max_iter=2000)

model = logisticRegr.fit(X_train, y_train)

y_pred_lr = model.predict(X_test)

# F1 Score on Logisitic Regression

In [ ]:
f1_score(y_test, y_pred_lr)

# Decision Tree Classifier

In [ ]:
d_tree= DecisionTreeClassifier(max_depth =3, random_state = 42)

model_tree = d_tree.fit(X_train, y_train)

In [ ]:
y_pred_dtree = model_tree.predict(X_test)

# F1 Score on Decision Tree Classifier

In [ ]:
f1_score(y_test, y_pred_dtree)

# Cat Boost Classifier

In [ ]:
cat_boost = cb.CatBoostClassifier(loss_function='CrossEntropy', iterations=5000, od_wait=100, od_type='Iter', depth=3)

model_cb = cat_boost.fit(X_train, y_train)

In [ ]:
y_pred_cb = model_cb.predict(X_test)

# F1 Score on Cat Boost Classifier

In [ ]:
f1_score(y_pred_cb, y_test)

# Over Sampling

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cancer',axis=1), df['cancer'], test_size=0.3)

In [ ]:
from imblearn.over_sampling import RandomOverSampler

os = RandomOverSampler(0.7)

X_train_os, y_train_os = os.fit_resample(X_train, y_train)

print('Before Sampling {}'.format(Counter(y_train)))
print('After Sampling {}'.format(Counter(y_train_os)))


In [ ]:
rand_for_classifier = RandomForestClassifier()
rand_for_classifier.fit(X_train_os, y_train_os)

y_pred_rf_clf = rand_for_classifier.predict(X_test)

print(confusion_matrix(y_pred_rf_clf, y_test))
print(classification_report(y_pred_rf_clf, y_test))

In [ ]:
count_class_0, count_class_1

# Creating a dataframe with equal number of target column values

In [ ]:
df_class_1_over = df_class_1.sample(count_class_0, replace=True)

df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

df_test_over['cancer'].value_counts()

In [ ]:
df_test_over.shape

In [ ]:
X = df_test_over.drop('cancer', axis=1)
y = df_test_over['cancer']

In [ ]:
X.shape, y.shape

# Separating the train and test features

In [ ]:
# stratify = y will ensure that the training and testing samples are balanced for both classes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

# Cat Boost Classifier

In [ ]:
cat_boost = cb.CatBoostClassifier(loss_function='CrossEntropy', iterations=5000, od_wait=100, od_type='Iter', depth=3)

model_cb = cat_boost.fit(X_train, y_train)

In [ ]:
y_pred_cb = model_cb.predict(X_test)

# F1 Score on Cat Boost Classifier

In [ ]:
f1_score(y_pred_cb, y_test)

# Decision Tree Classifier

In [ ]:
d_tree= DecisionTreeClassifier(max_depth =3, random_state = 42)

model_tree = d_tree.fit(X_train, y_train)

y_pred_dtree = model_tree.predict(X_test)

# F1 Score on Decision Tree Classifier

In [ ]:
f1_score(y_pred_dtree, y_test)

# Logisitic Regression

In [ ]:
logisticRegr = LogisticRegression(max_iter=2000)

model = logisticRegr.fit(X_train, y_train)

y_pred_lr = model.predict(X_test)

# F1 Score on Logistic Regression

In [ ]:
f1_score(y_pred_lr, y_test)

# Random Forest Classifier

In [ ]:
rand_forest = RandomForestClassifier()

model_randforest = rand_forest.fit(X_train, y_train)

y_pred = model_randforest.predict(X_test)

# F1 Score on Random Forest Classifier

In [ ]:
f1_score(y_pred, y_test)

# SMOTE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cancer', axis=1), df['cancer'], test_size=0.3, random_state=0)

In [ ]:
from imblearn.combine import SMOTETomek

st = SMOTETomek(0.75)

X_train_st, y_train_st = st.fit_resample(X_train, y_train)

print('Before Sampling {}'.format(Counter(y_train)))
print('After Sampling {}'.format(Counter(y_train_st)))

In [ ]:
rand_for_classifier = RandomForestClassifier()
rand_for_classifier.fit(X_train_st, y_train_st)

y_pred_rf_clf = rand_for_classifier.predict(X_test)

print(confusion_matrix(y_pred_rf_clf, y_test))
print(classification_report(y_pred_rf_clf, y_test))


In [ ]:
X = df.drop('cancer', axis=1)
y = df['cancer']

# Creating SMOTE Features for both independent and dependent feature

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')

X_smote, y_smote = smote.fit_resample(X,y)

In [ ]:
X_smote.shape, y_smote.shape

In [ ]:
y_smote.value_counts()

# Spliting the data in training and testing data

In [ ]:
# stratify = y will ensure that the training and testing samples are balanced for both classes
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.3, random_state=15, stratify=y_smote)

In [ ]:
'''
def build_model(hp):
    model = keras.Sequential()
    
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),
                                           min_value=32,
                                           max_value=512,
                                           step=32),
                              activation='relu'))
    model.add(keras.layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['binary_crossentropy'])
    return model'''

In [ ]:
'''tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5)'''

In [ ]:
#tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [ ]:
#tuner.results_summary()

In [ ]:
'''
def create_model(layers, activation):
    
    model = Sequential()
    
    for i, nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(nodes, input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model'''

In [ ]:
'''model = KerasClassifier(build_fn=create_model, verbose=0)

layers = [[20], [40, 20], [45, 30, 35]]
activation = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activation, batch_size=[128,256], epochs=[30])

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)'''

In [ ]:
#grid_result = grid.fit(X_train, y_train)

In [ ]:
#y_pred = grid.predict(X_test)
#print(confusion_matrix(y_pred, y_test))

In [ ]:
#print(grid_result.best_score_, grid_result.best_params_)

In [ ]:
#f1_score(y_pred, y_test)

In [ ]:
def ann(X_train, X_test, y_train, y_test):
    model = keras.Sequential(
            [
                keras.layers.Dense(X_train.shape[1], input_dim=X_train.shape[1], activation='relu'),
                keras.layers.Dense(45, activation='relu'),
                keras.layers.Dense(30, activation='relu'),
                keras.layers.Dense(35, activation='relu'),
                keras.layers.Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid'),
            ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
model = ann(X_train, X_test, y_train, y_test)

In [ ]:
model.fit(X_train, y_train, epochs=30)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = (y_pred>0.5)
f1_score(y_pred, y_test)

# Random Forest Classifier

In [ ]:
rand_forest = RandomForestClassifier(random_state=68, max_samples=0.5)

model_randforest = rand_forest.fit(X_train, y_train)

y_pred = model_randforest.predict(X_test)

In [ ]:
y_pred_train = model_randforest.predict(X_train)

# F1 Score on Random Forest Classifier

In [ ]:
f1_score(y_pred_train, y_train)

In [ ]:
f1_score(y_pred, y_test)

# Decision Tree Classifier

In [ ]:
d_tree= DecisionTreeClassifier()

model_tree = d_tree.fit(X_train, y_train)

y_pred_dtree = model_tree.predict(X_test)

# F1 Score on Decision Tree Classifier

In [ ]:
f1_score(y_pred_dtree, y_test)

# Cat Boost Classifier

In [ ]:
cat_boost = cb.CatBoostClassifier(loss_function='CrossEntropy', iterations=5000, od_wait=100, od_type='Iter', depth=3)

model_cb = cat_boost.fit(X_train, y_train)

In [ ]:
y_pred_cb = model_cb.predict(X_test)

# F1 Score on Cat Boost Classifier

In [ ]:
f1_score(y_pred_cb, y_test)